In [2]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install accelerate>=0.12.0
!pip install datasets
!pip install sentence_transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [ ]:
import json
import torch
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer

threshold = 0.25

model = SentenceTransformer('all-mpnet-base-v2', device='cuda' if torch.cuda.is_available() else "cpu")
# model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda' if torch.cuda.is_available() else "cpu")
raw_datasets = load_dataset("orkg/SciQA")
print(raw_datasets)
embed_data = torch.load('train_embeddings.pt')
dolly = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")


def divide_chunks(l_, n_):
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]


def save_json(filename, data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def get_similar(element, items=None, embeddings=None, num=None, reversed=False):
    emb_items = None

    if items is None and embeddings is not None:
        emb_items = embeddings["emb_questions"]
        items = embeddings["keys"]
    elif items is not None:
        emb_items = model.encode(items)

    if len(element) == 0 or emb_items is None:
        return []

    emb_element = model.encode(element)

    result = []
    scores = cos_sim(emb_element, emb_items)

    if num is None or num < 2:
        maximus = torch.max(scores, 1)
        m = float(maximus.values[0])
        i = int(maximus.indices[0])
        if m > threshold:
            result = [[round(m, 4), items[i], embeddings["questions"][i], embeddings["queries"][i]]]
        return result
    else:
        scored_texts = []
        for i, score in enumerate(scores[0]):
            scored_texts.append(
                [round(score.item(), 4), items[i], embeddings["questions"][i], embeddings["queries"][i]])
        sorted_scored_texts = sorted(scored_texts, key=lambda x: x[0], reverse=True)

        keys = []
        samples = []
        for sample in sorted_scored_texts:
            if sample[1] not in keys:
                samples.append(sample)
                keys.append(sample[1])

        samples = samples[:num]
        if reversed:
            samples.reversed()
        return samples


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def get_key(q):
    t0 = q.get('template_id')
    if t0 is None:
        t0 = "None"
    t = str(q.get("number_of_patterns")) + "-" + t0
    return t


def save_embedding():
    train = raw_datasets.get("train")
    questions = [q["question"]["string"] for q in train]
    queries = [clean(q["query"]["sparql"]) for q in train]
    keys = [get_key(q) for q in train]
    embeddings = {}
    emb_questions = model.encode(questions)
    embeddings["questions"] = questions
    embeddings["emb_questions"] = emb_questions
    embeddings["queries"] = queries
    embeddings["keys"] = keys
    torch.save(embeddings, 'train_embeddings.pt')
    return embeddings


def prepare_queries(n_, reversed=False):
    data = raw_datasets.get("test")
    queries = []
    suggestions = []
    for q in data:
        t = get_key(q)
        question = q["question"]["string"]
        suggestion = get_similar(question, embeddings=embed_data, num=n_, reversed=reversed)
        suggestions.append([[[x[0], x[1]] for x in suggestion], t])

        if suggestion is None or len(suggestion) == 0:
            print("Error with key", t)
            queries.append("translate the following English text '" + question + "' to a sparql query")
        else:
            final_q = ""
            for i_, k in enumerate(suggestion):
                final_q += "\n input (English text): " + k[2]
                final_q += "\n output (Sparql query): " + k[3]

            # works better with gpt
            # final_q += "\n with this example what is the sparql query for:  " + question

            # works better with dolly
            final_q += "\n input (English text): " + question
            final_q += "\n output (Sparql query): "
            queries.append(final_q)
    return queries, suggestions


def main(shots=7, attempts=10, batch=50, reversed=False):
    query_list, suggestions = prepare_queries(shots, reversed)
    print(len(query_list))

    n = batch
    q_list = list(divide_chunks(query_list, n))
    sparql = [clean(x["query"]["sparql"]) for x in raw_datasets.get("test")]

    gs = []
    lens = []
    i = 0

    for group in q_list:
        print(str(i) + "%", end="  ")
        i += 1 / len(q_list) * 100

        res_ = [tokenizer.encode(question) for question in group]
        len_ = [len(x) for x in res_]
        warning = [x for x in len_ if x > 2048]
        if len(warning) > 0:
            print(warning)
            quit()
        lens += len_

        res = dolly(group)
        print(res)
        gst = [x[0]["generated_text"] for x in res]

        for ii, l in enumerate(gst):
            for iii in range(attempts):
                if "SELECT" not in l:
                    print(iii, ii)
                    res = dolly(group[ii])
                    gst[ii] = res[0]["generated_text"]
                    l = gst[ii]
                else:
                    break
        gs += gst

        result = {"questions": query_list, "sparql": sparql, "generated_sparql": gs, "prompt_len": lens,
                  "suggestions": suggestions}
        save_json("test_A_nlp_dolly_" + str(shots) + "_shot_results_tok.json", result)

main()

DatasetDict({
    train: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 1795
    })
    validation: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 257
    })
    test: Dataset({
        features: ['id', 'query_type', 'question', 'paraphrased_question', 'query', 'template_id', 'query_shape', 'query_class', 'auto_generated', 'number_of_patterns'],
        num_rows: 513
    })
})
513
0%  [[{'generated_text': 'SELECT DISTINCT?model?model_lbl \n  WHERE {?metric a orkgc:Metric; rdfs:label?metric_lbl. FILTER (str(?metric_lbl) = "Accuracy") { SELECT?model?model_lbl WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "WSC")?benchmark orkgp:HAS_DATASET?dataset; orkgp

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0 10
1 10
0 13
1 13
2 13
3 13
4 13
0 14
0 15
1 15
2 15
3 15
4 15
5 15
6 15
7 15
8 15
9 15
0 16
0 21
0 25
0 26
1 26
0 28
1 28
0 29
0 30
1 30
2 30
3 30
4 30
5 30
6 30
7 30
8 30
9 30
0 33
1 33
2 33
3 33
4 33
5 33
6 33
7 33
8 33
9 33
0 34
1 34
0 35
1 35
2 35
0 37
1 37
2 37
0 39
1 39
2 39
3 39
0 43
0 45
1 45
0 46
1 46
2 46
0 47
1 47
0 49
9.090909090909092%  [[{'generated_text': 'SELECT DISTINCT?metric?metric_lbl (MAX(?value) AS?score) WHERE { { SELECT?metric?metric_lbl?value WHERE {?dataset a orkgc:Dataset; rdfs:label?dataset_lbl. FILTER (str(?dataset_lbl) = "AESLC")?benchmark orkgp:HAS_DATASET?dataset; orkgp:HAS_EVALUATION?eval.?eval orkgp:HAS_VALUE?value. OPTIONAL {?eval orkgp:HAS_METRIC?metric.?metric rdfs:label?metric_lbl. }?cont orkgp:HAS_BENCHMARK?benchmark. OPTIONAL {?cont orkgp:HAS_MODEL?model.?model rdfs:label?model_lbl. } } ORDER BY DESC(?value) } } GROUP BY?metric?metric_lbl'}], [{'generated_text': 'Please find attached a query to list the paper titles and IDs that include a benc